In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from datetime import datetime
from random import sample
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import os

In [3]:
# Load Files
drive_path = '/content/drive/My Drive/'
input_folder = drive_path + 'KaggleDatasets/RAW/'
output_folder = drive_path + 'KaggleDatasets/PRE/'
filename18 = 'preprocessado2018.csv'
filename19 = 'preprocessado2019.csv'

datasets18 = []
datasets19 = []
for entry in os.listdir(input_folder):
    file_path = os.path.join(input_folder, entry)
    ds = pd.read_csv(file_path)
    if entry.startswith("public") or entry.startswith("sample"):
        datasets19.append(ds)
    else:
        datasets18.append(ds)


In [4]:
dataset18 = pd.concat(datasets18, sort=False)
dataset19 = pd.concat(datasets19, sort=False)

print("dataset2018: ", dataset18.shape)
print("dataset2019: ", dataset19.shape)

dataset2018:  (28720, 7860)
dataset2019:  (13760, 7748)


In [5]:
dataset18['year'] = 2018
dataset19['year'] = 2019

In [6]:
dataset = pd.concat([dataset18, dataset19])
dataset.to_csv(output_folder + 'alldata.csv', index=False)

In [ ]:
dataset.to_csv(output_folder + 'alldata.csv', index=False)

In [ ]:
dataset = dataset.drop(['id'], axis=1)

In [ ]:
dayOfWeek={'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6}
dataset['date'] = dataset['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
dataset['date'] = dataset['date'].dt.day_name().map(dayOfWeek)
dataset

,date,input_0,input_1,input_2,input_3,input_4_1,input_5_1,input_6_1,input_7_1,input_8_1,input_9_1,input_10_1,input_11_1,input_12_1,input_13_1,input_14_1,input_15_1,input_16_1,input_17_1,input_18_1,input_19_1,input_20_1,input_21_1,input_22_1,input_23_1,input_24_1,input_25_1,input_26_1,input_27_1,input_28_1,input_29_1,input_30_1,input_31_1,input_32_1,input_33_1,input_34_1,input_35_1,input_36_1,input_37_1,input_38_1,...,output_10_4,output_11_4,output_12_4,output_13_4,output_14_4,output_15_4,output_16_4,output_1_5,output_2_5,output_3_5,output_4_5,output_5_5,output_6_5,output_7_5,output_8_5,output_9_5,output_10_5,output_11_5,output_12_5,output_13_5,output_14_5,output_15_5,output_16_5,output_1_6,output_2_6,output_3_6,output_4_6,output_5_6,output_6_6,output_7_6,output_8_6,output_9_6,output_10_6,output_11_6,output_12_6,output_13_6,output_14_6,output_15_6,output_16_6,year
0,0,9,0,213,62,0.667446,4.239122,0.535940,5.953665,2.599463,0.991817,0.950876,4.890664,0.670403,1.031335,-0.054883,-0.016268,-0.450593,-0.548683,0.025555,1.773580,-0.171833,-0.695826,0.431063,-0.254022,-0.872461,-0.240820,-0.749109,-0.773652,-0.660343,-0.201386,1.281555,0.958208,-0.791716,-0.083200,-0.712293,-0.686984,-0.457541,-0.044498,-0.183124,...,3.507291,1.834975,0.056275,-0.083200,2.611041,2.653370,3.502992,-0.029949,0.693608,-0.084863,0.355453,-0.240820,0.249804,-0.215169,0.818088,-0.201386,1.281555,0.081441,-0.155723,-0.083200,0.565912,-0.241604,0.532592,-0.029949,0.518578,0.648160,1.471738,-0.240820,-0.749109,0.064073,0.325277,0.397440,-0.295007,0.890764,-0.085057,-0.083200,-0.712293,-0.686984,-0.457541,2018
1,0,9,1,213,62,-0.146827,-0.208148,-0.044529,0.119220,1.811420,-0.426452,-0.224046,-0.262632,0.486903,-0.432452,-0.255508,-0.279677,-0.606179,-0.548683,-0.712166,0.038866,-0.065304,-0.029949,-0.356575,-0.648727,-0.760833,-0.240820,-0.749109,-0.773652,-0.660343,-0.201386,-0.480485,-0.323221,-0.862382,-0.083200,-0.712293,-0.686984,-0.457541,-0.044498,-0.183124,...,-0.109529,0.958208,-0.085057,-0.083200,0.054630,0.203777,0.532592,-0.695826,-0.356575,-0.535954,-0.537576,-0.240820,-0.416138,-0.773652,-0.167533,-0.201386,-0.387746,-0.525552,-0.367721,-0.083200,0.310271,-0.686984,-0.457541,-0.029949,-0.531605,-0.592340,-0.314318,-0.240820,-0.416138,-0.773652,-0.167533,-0.201386,-0.480485,-0.458109,-0.579718,-0.083200,-0.712293,-0.686984,-0.457541,2018
2,0,9,2,213,62,-0.749669,-0.208148,-0.624998,-0.610086,-0.552710,-0.426452,-0.876780,-0.644358,-0.797596,-0.432452,-0.295633,-0.411382,-0.761764,-0.548683,-0.846297,-0.770667,-0.917537,-0.695826,-0.969181,-1.043432,-0.984090,-0.240820,-0.416138,-0.773652,-0.660343,-0.201386,-0.573224,-0.997658,-1.003714,-0.083200,-0.712293,-0.686984,-0.457541,-0.044498,-0.183124,...,-0.665963,-0.795327,-0.650384,-0.083200,-0.712293,-0.464294,-0.457541,-0.695826,-0.706636,-0.874272,-0.537576,-0.240820,-0.416138,-0.773652,-0.660343,-0.201386,-0.851441,-0.862770,-0.721050,-0.083200,0.054630,-0.686984,-0.457541,-0.695826,-1.056697,-0.761500,-0.872461,-0.240820,-0.749109,-0.215169,-0.660343,-0.201386,-0.944180,-0.727883,-0.862382,-0.083200,-0.712293,-0.686984,-0.457541,2018
3,0,9,3,213,62,-0.739185,-0.208148,-0.624998,-0.610086,-0.552710,-0.426452,-0.876780,-0.835221,-0.797596,-0.432452,-0.295633,-0.411382,-0.761764,-0.548683,-0.980428,-0.770667,-0.917537,-0.029949,-1.056697,-0.930659,-0.984090,1.903065,-0.749109,-0.494411,-0.660343,-0.201386,-0.665963,-0.930214,-0.933048,-0.083200,-0.712293,-0.686984,-0.457541,-0.044498,-0.183124,...,-0.851441,-0.862770,-0.650384,-0.083200,-0.712293,-0.686984,-0.457541,-0.695826,-0.969181,-0.705113,-0.984090,-0.240820,-0.416138,-0.773652,-0.660343,-0.201386,-0.573224,-0.997658,-0.721050,-0.083200,-0.712293,-0.686984,-0.457541,-0.695826,-0.969181,-0.592340,-0.984090,-0.240820,-0.749109,0.064073,-0.660343,-0.201386,-0.944180,-0.660440,-0.791716,-0.083200,-0.712293,-0.686984,-0.457541,2018
4,0,16,0,179,137,1.189909,-0.158749,-0.059250,0.566645,0.673281,-0.302034,2.643322,2.137176,0.355893,0.549261,0.9732

In [ ]:
# Parâmetros
x = 55 # x%
y = 35 # y%
z = 20 # z%

# Tratamento
# NA > x% : descarta
# x% >NA > y% e variancia < z : descarta
# y% > NA: substitui pela media, moda ou mediana

total = len(dataset)
delete_list = []
median_list = []

variance = dataset.var()
mean = dataset.mean()

for column in dataset:
    if column.startswith("output"):
      continue

    num_na = dataset[column].isnull().sum()
    per_na = num_na * 100 / total
    aux = variance[column] * 100 / mean[column]
    if per_na > x:
        delete_list.append(column)
    elif per_na > y and aux > z:
        delete_list.append(column)
    elif num_na > 0:
        median_list.append(column)
        
dataset = dataset.drop(delete_list, axis=1)

median = dataset.median()
for column in median_list:
    dataset[column] = dataset[column].fillna(median[column])


    
dataset.to_csv(output_folder + filename, index=False)